In [2]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import math
from functools import partial
from open3d.t.geometry import TriangleMesh
import util

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
dataname = "/home/chris/Code/PointClouds/data/ply/CircularVentilationGrateExtraCleanedFull.ply"
pcd = o3d.io.read_point_cloud(dataname)
pcd = util.preProcessCloud(pcd)

In [4]:
def colorize_octree(octree, depth=8):
    colors = np.random.rand(2**depth, 3)  # Generate random colors for nodes

    def assign_color(node, node_info):
        if node_info.depth == depth:
            node.color = colors[node_info.child_index]  # Assign a color to each leaf node

    octree.traverse(assign_color)

# Define the octree depth
octree_depth = 8  # Higher depth = more subdivisions

# Create an octree and convert the point cloud into it
octree = o3d.geometry.Octree(max_depth=octree_depth)
octree.convert_from_point_cloud(pcd, size_expand=0.01)  # Expand slightly to ensure full coverage

colorize_octree(octree, depth=octree_depth)
o3d.visualization.draw_geometries([octree])


In [5]:
def calculate_eigen_norm_and_plane_direction(neighbor_coordinates):
    if len(neighbor_coordinates) < 3:
        return np.array([0, 0, 0]), np.array([0, 0, 0]), np.array([0, 0, 0])
    
    mu = np.mean(neighbor_coordinates, axis=0)
    norm = neighbor_coordinates - mu
    cov = np.cov(norm.T)
    eig_val, eig_vec = np.linalg.eig(cov)
    sorted_idx = np.argsort(eig_val)[::-1]
    eig_val = eig_val[sorted_idx]
    eig_vec = eig_vec[:, sorted_idx]
    eig_val_norm = eig_val.copy()

    for z in range(len(eig_val)):
        eig_val_norm[z] = np.exp(eig_val[z])/np.sum(np.exp(eig_val))

    plane_direction = np.cross(eig_vec[:, 0], eig_vec[:, 1])

    return mu, eig_val_norm, plane_direction

In [ ]:
def apply_pca_to_tree(pcd, octree, depth=8, vector_scale=2, min_points=10):
    lines = []
    colors = []
    all_points = np.asarray(pcd.points)
    def apply_pca(node, node_info):
        if node_info.depth == depth and isinstance(node, o3d.geometry.OctreeLeafNode):
            #print('Hi')
            if hasattr(node, "indices"):
                leaf_points = all_points[node.indices]

                if len(leaf_points) < min_points:
                    return
                
                mu, eig_val, plane_direction = calculate_eigen_norm_and_plane_direction(leaf_points)
                # Compute start and end points of the line
                start_point = mu - (vector_scale / 2) * plane_direction
                end_point = mu + (vector_scale / 2) * plane_direction

                # Store the line
                lines.append([start_point, end_point])
                colors.append([0, 0, 1])  # Blue lines

    octree.traverse(apply_pca)


Total lines rendered: 51057


TypeError: draw_geometries(): incompatible function arguments. The following argument types are supported:
    1. (geometry_list: List[open3d.cuda.pybind.geometry.Geometry], window_name: str = 'Open3D', width: int = 1920, height: int = 1080, left: int = 50, top: int = 50, point_show_normal: bool = False, mesh_show_wireframe: bool = False, mesh_show_back_face: bool = False) -> None
    2. (geometry_list: List[open3d.cuda.pybind.geometry.Geometry], window_name: str = 'Open3D', width: int = 1920, height: int = 1080, left: int = 50, top: int = 50, point_show_normal: bool = False, mesh_show_wireframe: bool = False, mesh_show_back_face: bool = False, lookat: numpy.ndarray[numpy.float64[3, 1]], up: numpy.ndarray[numpy.float64[3, 1]], front: numpy.ndarray[numpy.float64[3, 1]], zoom: float) -> None

Invoked with: LineSet with 51057 lines.